In [ ]:
%pip install -q gradio pandas

In [10]:
# STEP 2: IMPORT LIBRARIES

import pandas as pd
import gradio as gr
import random
from google.colab import files

In [11]:

# STEP 3: UPLOAD DATASET

print("=" * 60)
print("📂 Please upload your Cleaned_Indian_Food_Dataset.csv file")
print("=" * 60)

uploaded = files.upload()

# Load the uploaded file
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

print(f"\n✅ Dataset loaded successfully!")
print(f"📊 Total recipes: {len(df)}")
print(f"📋 Columns: {df.columns.tolist()}\n")

# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

📂 Please upload your Cleaned_Indian_Food_Dataset.csv file


Saving Cleaned_Indian_Food_Dataset.csv to Cleaned_Indian_Food_Dataset (1).csv

✅ Dataset loaded successfully!
📊 Total recipes: 5938
📋 Columns: ['TranslatedRecipeName', 'TranslatedIngredients', 'TotalTimeInMins', 'Cuisine', 'TranslatedInstructions', 'URL', 'Cleaned-Ingredients', 'image-url', 'Ingredient-count']



In [12]:
# STEP 4: AUTO-DETECT COLUMNS
# Find recipe name column
if 'recipename' in df.columns:
    name_col = 'recipename'
elif 'name' in df.columns:
    name_col = 'name'
elif 'recipe_name' in df.columns:
    name_col = 'recipe_name'
else:
    name_col = df.columns[0]

# Find ingredients column
if 'ingredients' in df.columns:
    ing_col = 'ingredients'
elif 'ingrediants' in df.columns:
    ing_col = 'ingrediants'
elif 'ingredient' in df.columns:
    ing_col = 'ingredient'
else:
    ing_col = df.columns[1]

# Find instructions column
if 'instructions' in df.columns:
    inst_col = 'instructions'
elif 'steps' in df.columns:
    inst_col = 'steps'
elif 'method' in df.columns:
    inst_col = 'method'
else:
    inst_col = None

print(f"🔍 Detected columns:")
print(f"   - Recipe Name: {name_col}")
print(f"   - Ingredients: {ing_col}")
print(f"   - Instructions: {inst_col if inst_col else 'Not found'}\n")

🔍 Detected columns:
   - Recipe Name: translatedrecipename
   - Ingredients: translatedingredients
   - Instructions: Not found



In [13]:
# STEP 5: SEARCH RECIPES FUNCTION
def search_recipes(user_ingredients, dietary_pref, cuisine_type):
    """Search and display matching recipes"""

    if not user_ingredients.strip():
        return "⚠️ **Please enter at least one ingredient!**"

    # Split and clean ingredients
    ing_list = [ing.strip().lower() for ing in user_ingredients.split(',')]

    # Calculate match scores
    def calc_score(recipe_ing):
        if pd.isna(recipe_ing) or recipe_ing == '':
            return 0
        recipe_ing = str(recipe_ing).lower()
        matches = sum(1 for ing in ing_list if ing in recipe_ing)
        return matches

    df['score'] = df[ing_col].apply(calc_score)

    # Filter recipes with at least 1 match
    filtered = df[df['score'] > 0].copy()

    # Apply dietary filter
    if dietary_pref == "Vegetarian" and 'diet' in df.columns:
        filtered = filtered[filtered['diet'].str.contains('veg', case=False, na=False)]
    elif dietary_pref == "Non-Vegetarian" and 'diet' in df.columns:
        filtered = filtered[filtered['diet'].str.contains('non', case=False, na=False)]

    # Apply cuisine filter
    if cuisine_type != "Any" and 'cuisine' in df.columns:
        filtered = filtered[filtered['cuisine'].str.contains(cuisine_type, case=False, na=False)]

    # Get top 3 matches
    top_recipes = filtered.nlargest(3, 'score')

    if len(top_recipes) == 0:
        return "😔 **No matching recipes found!**\n\nTry:\n- Different ingredients\n- Less specific filters\n- Common ingredients like rice, dal, chicken"

    # Build result
    result = f"# 🎯 Found {len(top_recipes)} Perfect Recipes!\n\n"
    result += "---\n\n"

    for idx, (i, row) in enumerate(top_recipes.iterrows(), 1):
        recipe_name = row[name_col]
        ingredients = row[ing_col]
        instructions = row[inst_col] if inst_col and not pd.isna(row[inst_col]) else "Follow traditional cooking method for this recipe."

        result += f"## 🍽️ Recipe {idx}: **{recipe_name}**\n\n"
        result += f"**📊 Match Score:** {int(row['score'])} of your ingredients matched!\n\n"

        # Ingredients section
        result += f"### 📝 Ingredients:\n"
        result += f"```\n{ingredients}\n```\n\n"

        # Instructions section
        result += f"### 👨‍🍳 Cooking Instructions:\n"

        if isinstance(instructions, str) and len(instructions) > 50:
            # Split instructions into steps
            steps = instructions.replace('\n', '. ').split('.')
            step_num = 1
            for step in steps[:10]:  # Limit to 10 steps
                step = step.strip()
                if len(step) > 10:  # Only meaningful steps
                    result += f"{step_num}. {step}.\n"
                    step_num += 1
        else:
            result += f"{instructions}\n"

        # Add cooking tips
        result += f"\n### 💡 Pro Tips:\n"
        all_tips = [
            "🔪 Prep all ingredients before you start cooking",
            "🔥 Use medium heat for better control",
            "🧂 Taste and adjust seasoning at the end",
            "🌿 Fresh ingredients always taste better",
            "⏰ Don't rush - let flavors develop",
            "🥄 Stir occasionally to prevent burning"
        ]
        selected_tips = random.sample(all_tips, 2)
        for tip in selected_tips:
            result += f"- {tip}\n"

        # Additional info if available
        if 'prep_time' in df.columns and not pd.isna(row['prep_time']):
            result += f"\n**⏱️ Prep Time:** {row['prep_time']}\n"
        if 'cook_time' in df.columns and not pd.isna(row['cook_time']):
            result += f"**⏱️ Cook Time:** {row['cook_time']}\n"

        result += "\n---\n\n"

    return result

In [16]:
# STEP 6: CREATE BEAUTIFUL UI

with gr.Blocks(theme=gr.themes.Soft(primary_hue="orange", secondary_hue="amber")) as app:

    gr.Markdown("""
    # 🍳 AI Recipe Preparation Agent
    ### 🇮🇳 Discover Delicious Indian Recipes from Your Ingredients!
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🎯 Enter Your Details")

            ingredients = gr.Textbox(
                label="🥘 Available Ingredients",
                placeholder="Example: rice, tomato, onion, green chili, spices",
                lines=3,
                info="Separate multiple ingredients with commas"
            )

            dietary = gr.Radio(
                choices=["Any", "Vegetarian", "Non-Vegetarian"],
                label="🥗 Dietary Preference",
                value="Any",
                info="Filter by diet type"
            )

            cuisine = gr.Dropdown(
                choices=["Any", "North Indian", "South Indian", "Bengali", "Punjabi", "Gujarati"],
                label="🌍 Cuisine Type",
                value="Any",
                info="Choose regional cuisine"
            )

            search_btn = gr.Button("🔍 Find My Recipes", variant="primary", size="lg")

            gr.Markdown("""
            ---
            #### 💡 Quick Examples:
            - Rice dishes: *rice, dal, turmeric*
            - Chicken curry: *chicken, yogurt, spices*
            - Paneer: *paneer, tomato, cream*
            """)

        with gr.Column(scale=2):
            gr.Markdown("### 📖 Your Recipe Results")
            output = gr.Markdown(value="*Enter ingredients and click 'Find My Recipes' to start!*")

    # Example recipes
    gr.Markdown("### 🌟 Try These Popular Searches:")
    gr.Examples(
        examples=[
            ["rice, dal, turmeric, onion", "Vegetarian", "Any"],
            ["chicken, yogurt, ginger, garlic, spices", "Non-Vegetarian", "North Indian"],
            ["paneer, tomato, cream, butter", "Vegetarian", "Punjabi"],
            ["potato, cauliflower, turmeric, cumin", "Vegetarian", "Any"],
            ["fish, mustard, coconut", "Non-Vegetarian", "Bengali"],
        ],
        inputs=[ingredients, dietary, cuisine],
        label="Click any example to try it!"
    )

    # Connect button to function
    search_btn.click(
        fn=search_recipes,
        inputs=[ingredients, dietary, cuisine],
        outputs=output
    )

    # Footer
    gr.Markdown(f"""
    ---
    ### 📊 Dataset Statistics
    - Total Recipes Available: {len(df):,}
    - Columns in Dataset: {', '.join(df.columns.tolist()[:5])}...
    - Source: Cleaned Indian Food Dataset

    """)

In [17]:
# STEP 7: LAUNCH THE APP

print("\n" + "=" * 60)
print("🚀 LAUNCHING RECIPE FINDER APP...")
print("=" * 60 + "\n")

print("✨ App is starting...")
print("📱 You'll get a public link to share with anyone!")
print("🔗 Link will be valid for 72 hours\n")

app.launch(share=True, debug=False)

print("\n✅ App is now LIVE!")
print("💡 Tip: Bookmark the link to access later!")


🚀 LAUNCHING RECIPE FINDER APP...

✨ App is starting...
📱 You'll get a public link to share with anyone!
🔗 Link will be valid for 72 hours

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://16ebd692370985ddb6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



✅ App is now LIVE!
💡 Tip: Bookmark the link to access later!
